In [ ]:
# !conda install --yes --prefix {sys.prefix} scikit-image
# !pip3 install scikit-image

In [13]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import torch.utils.data
from skimage import io, transform
from skimage.measure import block_reduce
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import argparse

In [16]:
# Training settings
parser = argparse.ArgumentParser(description='Deep Learning JHU Assignment 1 - Fashion-MNIST')
parser.add_argument('--batch-size', type=int, default=256, metavar='B',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='TB',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='E',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--optimizer', type=str, default='sgd', metavar='O',
                    help='Optimizer options are sgd, p1sgd, adam, rms_prop')
parser.add_argument('--momentum', type=float, default=0.5, metavar='MO',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log_interval', type=int, default=100, metavar='I',
                    help="""how many batches to wait before logging detailed
                            training status, 0 means never log """)
parser.add_argument('--dataset', type=str, default='mnist', metavar='D',
                    help='Options are mnist and fashion_mnist')
parser.add_argument('--data_dir', type=str, default='../data/', metavar='F',
                    help='Where to put data')
parser.add_argument('--name', type=str, default='', metavar='N',
                    help="""A name for this training run, this
                            affects the directory so use underscores and not spaces.""")
parser.add_argument('--model', type=str, default='default', metavar='M',
                    help="""Options are default, P2Q7DefaultChannelsNet,
                    P2Q7HalfChannelsNet, P2Q7DoubleChannelsNet,
                    P2Q8BatchNormNet, P2Q9DropoutNet, P2Q10DropoutBatchnormNet,
                    P2Q11ExtraConvNet, P2Q12RemoveLayerNet, and P2Q13UltimateNet.""")
parser.add_argument('--print_log', action='store_true', default=False,
                    help='prints the csv log when training is complete')
parser.add_argument('--dropout_rate', type=float, default=0.5)
parser.add_argument("-f", "--jupyter-json")


required = object()
args = parser.parse_args()

In [2]:
def get_photo_file(photo_name):
    return "data/yelp_photos/photos/" + photo_name + ".jpg"

def resize_img(img):
    return transform.resize(img, (224, 224))

In [21]:
class YelpDataset(torch.utils.data.Dataset):
    
    def __init__(self, df, batch_size, shuffle=True):
        self.df = df 
        self.batch_size = batch_size
        if shuffle:
            self.df = self.df.sample(frac=1)
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        photo_names = self.df.iloc[idx:idx+self.batch_size].index.tolist()
        file_names = [get_photo_file(photo_name) for photo_name in photo_names]
        images = [io.imread(file_name) for file_name in file_names]
        images = [resize_img(img) for img in images] 
        label = self.df.iloc[idx:idx+self.batch_size].label.tolist()
        return (torch.Tensor(images),torch.Tensor(label))

In [4]:
biz_df = pd.read_csv("data/clean_business.csv").set_index("business_id")
photo_df = pd.read_csv("data/clean_photo.csv").set_index("photo_id")
df = photo_df.copy(deep=True)
df["label"] = pd.Series(biz_df.loc[df["business_id"]]["stars"]).tolist()
df = df.sample(frac=1)

# train_df = df.iloc[0:int(len(df) * 0.7)]
# val_df = df.iloc[int(len(df) * 0.7):]

val_df = df.iloc[0:1000]
train_df = df.iloc[1000:11000]

In [5]:
train_loader = YelpDataset(train_df, batch_size=args.batch_size)
val_loader = YelpDataset(val_df, batch_size=len(val_df))

In [6]:
val_img, val_label = val_loader[0]
val_img = Variable(val_img)
val_labeel = Variable(val_label)

/usr/local/opt/python3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [7]:
class BasicNet(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(BasicNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, kernel_size=5)
        self.conv2 = nn.Conv2d(5, 10, kernel_size=5)
        self.fc1 = nn.Linear(28090, 100)
        self.fc2 = nn.Linear(100, 1)
        self.dropout_rate = dropout_rate

    def forward(self, x):
        x = x.permute(0, 3, 1, 2)
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 28090)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=self.dropout_rate, training=self.training)
        x = self.fc2(x)
        return x

In [8]:
model = BasicNet()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [9]:
def train_step(step):

    model.train()

    input_batch, label_batch = train_loader[step]
    input_batch = Variable(input_batch)
    label_batch = Variable(label_batch)
    output_batch = model(input_batch)

    loss = F.l1_loss(output_batch, label_batch)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
    return loss.data[0]

def train_epoch():
    
    model.train()
    i = 0
    loss_list = []
    for input_batch, label_batch in train_loader:
    
        input_batch = Variable(input_batch)
        label_batch = Variable(label_batch)
        output_batch = model(input_batch)

        loss = F.l1_loss(output_batch, label_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print("Training step " + str(i) + " " + str(loss.data[0]))
        i += 1
        loss_list.append(loss)
        if i == 10:
            break
    
    total_loss = 0
    for loss in loss_list:
        total_loss += loss
    total_loss /= i

    return total_loss, loss_list

In [10]:
for module in model.children():
    module.reset_parameters()
    
avg_loss, loss_list = train_epoch()

/usr/local/opt/python3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Training step 0 3.571491241455078
Training step 1 3.5412356853485107
Training step 2 3.57373309135437
Training step 3 3.5447933673858643
Training step 4 3.472736358642578
Training step 5 3.4579105377197266
Training step 6 3.478287935256958
Training step 7 3.4323363304138184
Training step 8 3.334973096847534
Training step 9 3.239255428314209


In [11]:
# avg_loss

Variable containing:
 3.4647
[torch.FloatTensor of size 1]

In [20]:
# for module in model.children():
#     module.reset_parameters()

# info = []
# num_steps = 100
# for step in range(num_steps):
#     train_loss = train_step(step)
#     print(train_loss)
#     info.append(train_loss)

In [17]:
# plt.plot(range(num_steps), info)

In [18]:
# model.eval()
# pred = model(val_img[0:30])

In [19]:
# pred - Variable(val_label[0:30].view(30, 1))

In [15]:
# !conda install --yes --prefix {sys.prefix} scikit-image
# !pip3 install scikit-image

import sys
import pandas as pd
import matplotlib.pyplot as plt
import torch.utils.data
from skimage import io, transform
from skimage.measure import block_reduce
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import argparse


parser = argparse.ArgumentParser(description='Deep Learning JHU Assignment 1 - Fashion-MNIST')
parser.add_argument('--batch-size', type=int, default=256, metavar='B',
					help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='TB',
					help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='E',
					help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
					help='learning rate (default: 0.01)')
parser.add_argument('--optimizer', type=str, default='sgd', metavar='O',
					help='Optimizer options are sgd, p1sgd, adam, rms_prop')
parser.add_argument('--momentum', type=float, default=0.5, metavar='MO',
					help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
					help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
					help='random seed (default: 1)')
parser.add_argument('--log_interval', type=int, default=100, metavar='I',
					help="""how many batches to wait before logging detailed
							training status, 0 means never log """)
parser.add_argument('--dataset', type=str, default='mnist', metavar='D',
					help='Options are mnist and fashion_mnist')
parser.add_argument('--data_dir', type=str, default='data/', metavar='F',
					help='Where to put data')
parser.add_argument('--photos_dir', type=str, default='data/yelp_photos/photos/', metavar='F',
					help='Where to put photos')
parser.add_argument('--name', type=str, default='', metavar='N',
					help="""A name for this training run, this
							affects the directory so use underscores and not spaces.""")
parser.add_argument('--model', type=str, default='default', metavar='M',
					help="""Options are default, P2Q7DefaultChannelsNet,
					P2Q7HalfChannelsNet, P2Q7DoubleChannelsNet,
					P2Q8BatchNormNet, P2Q9DropoutNet, P2Q10DropoutBatchnormNet,
					P2Q11ExtraConvNet, P2Q12RemoveLayerNet, and P2Q13UltimateNet.""")
parser.add_argument('--print_log', action='store_true', default=False,
					help='prints the csv log when training is complete')
parser.add_argument('--dropout_rate', type=float, default=0.5)
parser.add_argument("-f", "--jupyter-json")
parser.add_argument('--loss', type=str, default='l1', metavar='LOSS',
					help='l1, mse')



required = object()
args = parser.parse_args()


def get_photo_file(photo_name):
	return args.photos_dir + photo_name + ".jpg"

def resize_img(img):
	return transform.resize(img, (224, 224))





In [29]:
class YelpDataset(torch.utils.data.Dataset):
	def __init__(self, df):
		self.df = df 

	def __len__(self):
		return len(self.df)

	def __getitem__(self, idx):
		photo_name = self.df.iloc[idx].name
		file_name = get_photo_file(photo_name)
		img = io.imread(file_name)
		img = resize_img(img) 
		label = self.df.iloc[idx].label
		return (torch.Tensor(img), torch.Tensor([label]))

biz_df = pd.read_csv(args.data_dir + "clean_business.csv").set_index("business_id")
photo_df = pd.read_csv(args.data_dir + "clean_photo.csv").set_index("photo_id")
df = photo_df.copy(deep=True)
df["label"] = pd.Series(biz_df.loc[df["business_id"]]["stars"]).tolist()
df = df.sample(frac=1)

# train_df = df.iloc[0:int(len(df) * 0.7)]
# val_df = df.iloc[int(len(df) * 0.7):]

val_df = df.iloc[0:1000]
train_df = df.iloc[1000:11000]


train_loader = YelpDataset(train_df)
val_loader = YelpDataset(val_df)


In [30]:
train_loader[0]

/usr/local/opt/python3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


(
 ( 0 ,.,.) = 
   0.1647  0.2314  0.2706
   0.1647  0.2314  0.2706
   0.1647  0.2314  0.2706
            ⋮            
   0.3098  0.3765  0.4039
   0.3098  0.3765  0.4039
   0.3098  0.3765  0.4039
 
 ( 1 ,.,.) = 
   0.1647  0.2314  0.2706
   0.1647  0.2314  0.2706
   0.1647  0.2314  0.2706
            ⋮            
   0.3105  0.3771  0.4046
   0.3105  0.3771  0.4046
   0.3105  0.3771  0.4046
 
 ( 2 ,.,.) = 
   0.1647  0.2314  0.2706
   0.1647  0.2314  0.2706
   0.1647  0.2314  0.2706
            ⋮            
   0.3137  0.3804  0.4078
   0.3137  0.3804  0.4078
   0.3137  0.3804  0.4078
 ... 
 
 (221,.,.) = 
   0.0390  0.0743  0.1018
   0.0390  0.0743  0.1018
   0.0390  0.0743  0.1018
            ⋮            
   0.0275  0.0706  0.0941
   0.0314  0.0745  0.0980
   0.0353  0.0784  0.1020
 
 (222,.,.) = 
   0.0353  0.0706  0.0980
   0.0353  0.0706  0.0980
   0.0353  0.0706  0.0980
            ⋮            
   0.0275  0.0706  0.0941
   0.0314  0.0745  0.0980
   0.0353  0.0784  0.1020
 
 (

In [22]:
photo_name = df.iloc[0].name
file_name = get_photo_file(photo_name)
img = io.imread(file_name)
img = resize_img(img) 
label = df.iloc[0].label

/usr/local/opt/python3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [24]:
torch.Tensor(img)


( 0 ,.,.) = 
  0.0980  0.0431  0.0078
  0.0980  0.0431  0.0078
  0.0980  0.0431  0.0078
           ⋮            
  0.5098  0.4392  0.3608
  0.5059  0.4353  0.3569
  0.5020  0.4314  0.3529

( 1 ,.,.) = 
  0.0980  0.0431  0.0078
  0.0980  0.0431  0.0078
  0.0980  0.0431  0.0078
           ⋮            
  0.5098  0.4385  0.3622
  0.5059  0.4346  0.3583
  0.5020  0.4307  0.3543

( 2 ,.,.) = 
  0.0980  0.0431  0.0078
  0.0980  0.0431  0.0078
  0.0980  0.0431  0.0078
           ⋮            
  0.5098  0.4353  0.3686
  0.5059  0.4314  0.3647
  0.5020  0.4275  0.3608
... 

(221,.,.) = 
  0.0235  0.0078  0.0039
  0.0235  0.0078  0.0039
  0.0235  0.0078  0.0039
           ⋮            
  0.4704  0.4038  0.3646
  0.4704  0.4038  0.3646
  0.4704  0.4038  0.3646

(222,.,.) = 
  0.0235  0.0078  0.0039
  0.0235  0.0078  0.0039
  0.0235  0.0078  0.0039
           ⋮            
  0.4634  0.3968  0.3576
  0.4634  0.3968  0.3576
  0.4634  0.3968  0.3576

(223,.,.) = 
  0.0235  0.0078  0.0039
  0.0235  0

In [27]:
torch.Tensor([label])


 3
[torch.FloatTensor of size 1]